In [1]:
#--- NMC622 Battery Project:


'''Research Based on:
Comparative life cycle assessment of sodium-ion and lithium iron phosphate batteries,
in the context of carbon neutrality.

2023. Wei Guo, Tao Feng, Wei Li, Lin Hua, Zhenghua Meng, Ke Li.

https://doi.org/10.1016/j.est.2023.108589

'''

#- 1st use: EV (Electric Vehicle)
#- 2nd use: CBS (communication base station)

'Research Based on:\nComparative life cycle assessment of sodium-ion and lithium iron phosphate batteries,\nin the context of carbon neutrality.\n\n2023. Wei Guo, Tao Feng, Wei Li, Lin Hua, Zhenghua Meng, Ke Li.\n\nhttps://doi.org/10.1016/j.est.2023.108589\n\n'

In [2]:
#-Inventory for EV usage:

VM = 1625	#kg, Vehicle_mass
L = 200_000	#km, Total_mileage
BM = 558	#kg, Battery_mass
Battery_capacity = 57	#kwh
Battery_energydensity = 102	#Wh/kg
Batterycell_energydensity =	160	#Wh/kg
Ep= 12.2	#kWh/100km , Electricity_consumption
Battery_chargedischarge_eff = 90	#%


In [19]:
#-  Electricity loss, EV phase:
# Em = f*Ep*L*(BM/VM)
# Ec = Ep*L*(1-n1) , where n1 is charge and discharge efficiency of the battery serving, 90%.

Ec= 2440	#kWh, 
Em= 4414.52 #kWh, Energy loss
f= 0.005268 #N/A, energy distribution coefficient of the battery

Elec_lossEV= 6854.52	#kWh, Total electricity loss
Elec_loss_kgEV= 11.42	#kWh/kg battery, Electricity loss per kg

#- Electricity net consumption over the EV lifetime (200 000 km)
Ep_totEV=(Ep/100)*L #kWh
Ep_netEV=Ep_totEV+Elec_lossEV #kWh

print(Ep_netEV)

31254.52


In [20]:
#-  Electricity loss, CBS phase:

Elec_lossCBS= 8773.22#kWh, Total electricity loss
Elec_loss_kgCBS= 14.62	#kWh/kg battery, Electricity loss per kg

#- Electricity net consumption over the CBS lifetime (200 000 km)
Ep_totCBS=(Ep/100)*L #kWh
Ep_netCBS=Ep_totCBS+Elec_lossCBS #kWh
print(Ep_netCBS)

33173.22


In [5]:
#- Importing libraries

import bw2io, bw2data, bw2calc

import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python

In [6]:
bw2data.projects

Brightway2 projects manager with 16 objects:
	Battery_Project
	Ecoinvent_3.10
	Parametric_LCA
	Temporalis example project
	Temporalis_Car
	bw25_intro
	bw2_example_16042024
	bw2_example_18042024
	bw2_example_Arrays_Matrices
	bw2_example_LCA_ei_Access
	bw_temporalis example
	dds-MFA
	default
	hydrogen
	parametric-LCA
	parametric_LCA
Use `projects.report()` to get a report on all projects.

In [9]:
#- Create a new project

#- To create a project: 
bw2data.projects.set_current("Parametric_LCA")

#- To copy a project with the Database
#bw2data.projects.copy_project('Battery_Project2', switch=True)


In [15]:
#- To see the databses
bw2data.databases

Databases dictionary with 6 object(s):
	biosphere3
	ecoinvent-3.10-cutoff
	energy_storage_analysis
	h2_electrolysis
	parametric_LCA
	super_db_12-11-2024

In [16]:
#- Excel importer

imp = bw2io.ExcelImporter(r'C:\Users\luis.carmona\Desktop\Work\5. Brightway\Autumn School - Nov 2024\Workshop repository\Battery Project\BP_energy_storage_analysis.xlsx')
imp.apply_strategies()


Extracted 1 worksheets in 0.11 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 11.82 seconds


In [21]:
#- Match databases:
imp.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
imp.statistics()

Applying strategy: link_iterable_by_fields
7 datasets
23 exchanges
2 unlinked exchanges
  Type technosphere: 1 unique unlinked exchanges


(7, 23, 2)

In [18]:
imp.write_excel()

Wrote matching file to:
C:\Users\luis.carmona\AppData\Local\pylca\Brightway3\Parametric_LCA.d435c90135103ef448cbd5acc32c6351\output\db-matching-energy_storage_analysis.xlsx


In [14]:
[u for u in imp.unlinked if u["type"] == "technosphere"]

[{'name': 'market for battery, Li-ion, NMC622',
  'amount': 5.70776255707762,
  'location': 'GLO',
  'unit': 'kilogram',
  'database': 'super_db_12-11-2024',
  'type': 'technosphere',
  'reference product': 'battery, Li-ion, NMC622',
  'comment': 'economic inflow'}]

In [13]:
imp.write_database()

Writing activities to SQLite3 database:
0% [#      ] 100% | ETA: 00:00:01

InvalidExchange: 

In [ ]:
#- New database

# relink foreground database to new background database

old_db_name = "BP"

# current background database
old_dependency_name = "ecoinvent-3.10-cutoff"

# new background database to link to
new_dependency_name = "lci-batteries-NMC622-LTO"

# new foreground database name
new_db_name = f"{old_db_name} {new_dependency_name}"

# load database object
db = bw2data.Database(old_db_name)

# turn it into a dictionary
data = db.load()

for i, ds in data.items():

    ds["database"] = new_db_name
    for e in ds["exchanges"]:
        #print(e)
        if e.get("input")[0] == old_dependency_name:
            e["input"] = (new_dependency_name, e["input"][1])
        
        e["output"] = (new_db_name, e["output"][1])

        if e["type"] == "production":
            e["output"] = e["input"]
data = {(new_db_name, k[1]): v for k, v in data.items()}

# write it back to project
bw2data.Database(new_db_name).write(data)

In [103]:
eidb = bw2data.Database('super_db_12-11-2024')

In [46]:
#--- LCA

In [ ]:
#- Emissions from the production of the battery

In [47]:
#- Defining Activity: Choosing the NMC622 Battery production:

Bat_nmc = [act for act in eidb if 'battery production' in act['name']
                        and 'NMC811' in act['name']
                        and 'RoW' in act['location']
            ][0]

In [48]:
print(Bat_nmc)

'battery production, Li-ion, NMC811, rechargeable, prismatic' (kilogram, RoW, None)


In [49]:
#- Defining the method:

ipcc2021 = [m for m in bw2data.methods if 'IPCC' in m[0]
                    and ('2021') in str(m)
                    and 'GWP100' in str(m)
                    and 'no LT' not in str(m)
                    and 'SLCF' not in str(m)
           ][0]

ipcc_2021_method = bw2data.Method(ipcc2021)

In [68]:
#ipcc_2021_method.load()

In [69]:
Bat_pdt = bw2calc.LCA({Bat_nmc:1}, ipcc_2021_method.name)

In [70]:
Bat_pdt.lci()    # Builds matrices, solves the system, generates an LCI matrix.
Bat_pdt.lcia()   # Characterization, i.e. the multiplication of the elements 
                          # of the LCI matrix with characterization factors from the chosen method
Bat_pdt.score    # Returns the score, i.e. the sum of the characterized inventory

16.22091021566939

In [71]:
#- Emissions from the usage: 1st phase (EV)

In [88]:
EV_Elec_need = [act for act in eidb if 'electricity' in act['name']
                        and 'market' in act['name']
                        and 'low voltage' in act['name']
                        and 'DE' in act['location']
            ][0]
print(EV_Elec_need)

'market for electricity, low voltage' (kilowatt hour, DE, None)


In [73]:
Bat_1st = bw2calc.LCA({EV_Elec_need:Ep_netEV}, ipcc_2021_method.name)

In [74]:
Bat_1st.lci()    # Builds matrices, solves the system, generates an LCI matrix.
Bat_1st.lcia()   # Characterization, i.e. the multiplication of the elements 
                          # of the LCI matrix with characterization factors from the chosen method
Bat_1st.score    # Returns the score, i.e. the sum of the characterized inventory

6890.096970890689

In [75]:
#- Emissions from the usage: 2ndt phase (CBS)

In [76]:
Bat_2nd = bw2calc.LCA({EV_Elec_need:Ep_netCBS}, ipcc_2021_method.name)

In [77]:
Bat_2nd.lci()    # Builds matrices, solves the system, generates an LCI matrix.
Bat_2nd.lcia()   # Characterization, i.e. the multiplication of the elements 
                          # of the LCI matrix with characterization factors from the chosen method
Bat_2nd.score    # Returns the score, i.e. the sum of the characterized inventory

6136.62490526192

In [78]:
#- The characterization matrix:

# Matrix of characterization factors:
Bat_2nd.characterization_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 93 stored elements and shape (2648, 2648)>

In [80]:
# Matrix of characterized inventory flows
Bat_2nd.characterized_inventory

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 19966 stored elements and shape (2648, 23523)>

In [85]:
# Create array with the results per column (i.e. per activity)
results_by_activity = (Bat_2nd.characterized_inventory.sum(axis=0)).A1

In [86]:
rev_act_dict, rev_prod_dict, rev_bio_dict = Bat_2nd.reverse_dict()
# Create a list of names in columns
list_of_names_in_columns = [
    bw2data.get_activity(rev_act_dict[col])['name'] 
    for col in range(Bat_2nd.characterized_inventory.shape[1])
]

In [87]:
pd.Series(index=list_of_names_in_columns, data=results_by_activity).sort_values(ascending=False).head(10)

electricity production, lignite                                                          2786.738997
electricity production, hard coal                                                         766.131002
heat and power co-generation, natural gas, conventional power plant, 100MW electrical     455.684639
heat and power co-generation, hard coal                                                   209.280176
electricity production, natural gas, conventional power plant                             184.479800
electricity production, natural gas, combined cycle power plant                           174.816023
natural gas venting from petroleum/natural gas production                                 113.941647
heat and power co-generation, lignite                                                      80.481114
transport, pipeline, onshore, long distance, natural gas                                   51.828479
heat production, anthracite, at stove 5-15kW                                               